### Importar librerias

In [1]:
import pandas as pd
import queue

### Leer dataset

In [2]:
data = pd.read_csv("Titanic.csv")

### Estructura del árbol

In [4]:
class Arbol:
    def __init__(self, raiz):
        self.hijos = []
        self.raiz = raiz


def agregarElemento(arbol, nodo, nodoPadre):
    subarbol = buscarSubarbol(arbol, nodoPadre)
    subarbol.hijos.append(Arbol(nodo))


def buscarSubarbol(arbol, nodo):
    if arbol.raiz == nodo:
        return arbol
    for subarbol in arbol.hijos:
        arbolBuscado = buscarSubarbol(subarbol, nodo)
        if (arbolBuscado != None):
            return arbolBuscado
    return None  


def profundidad(arbol):
    if len(arbol.hijos) == 0: 
        return 1
    return 1 + max(map(profundidad,arbol.hijos)) 


def grado(arbol, num):
    num += 1
    for hijo in arbol.hijos:
        grado(hijo, num)


def printArbolProfundidad(arbol, level = 0):
    print("|" + "-" * level * 20 + " " + arbol.raiz)
    for hijo in arbol.hijos:
        printArbolProfundidad(hijo, level+1)


def printArbolAnchura(arbol, cola):
    print(arbol.raiz)
    if(len(arbol.hijos) > 0):
        for hijo in arbol.hijos:
            cola.put(hijo)
    if(cola.qsize() != 0):
        printArbolAnchura(cola.get(), cola)

In [5]:
abuela = "Jacqueline Gurney"
marge = "Marge Bouvier"
patty = "Patty Bouvier"
selma = "Selma Bouvier"
bart = "Bart Simpson"
lisa = "Lisa Simpson"
maggie = "Maggie Simpson"
ling = "Ling Bouvier"

arbol = Arbol(abuela)
#agregarElemento(arbol, patty, abuela)
#agregarElemento(arbol, selma, abuela)
#agregarElemento(arbol, ling, selma)
#agregarElemento(arbol, marge, abuela)
#agregarElemento(arbol, bart, marge)
#agregarElemento(arbol, lisa, marge)
#agregarElemento(arbol, maggie, marge)

#print(profundidad(arbol))
printArbolProfundidad(arbol)
#cola = queue.Queue()
#printArbolAnchura(arbol, cola)


AttributeError: 'Arbol' object has no attribute 'agregarElemento'